In [6]:
import pandas as pd
from tqdm import tqdm

train = pd.read_csv('../input/feedback-prize-2021/train.csv')
train.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [7]:
classes = train.discourse_type.unique().tolist()
classes

['Lead',
 'Position',
 'Evidence',
 'Claim',
 'Concluding Statement',
 'Counterclaim',
 'Rebuttal']

In [8]:
from collections import defaultdict
tags = defaultdict()

for i, c in enumerate(classes):
    tags[f'B-{c}'] = i
    tags[f'I-{c}'] = i + len(classes)
tags[f'O'] = len(classes) * 2
tags[f'Special'] = -100
    
l2i = dict(tags)

i2l = defaultdict()
for k, v in l2i.items(): 
    i2l[v] = k
i2l[-100] = 'Special'

i2l = dict(i2l)

N_LABELS = len(i2l) - 1 # not accounting for -100

In [9]:
l2i

{'B-Lead': 0,
 'I-Lead': 7,
 'B-Position': 1,
 'I-Position': 8,
 'B-Evidence': 2,
 'I-Evidence': 9,
 'B-Claim': 3,
 'I-Claim': 10,
 'B-Concluding Statement': 4,
 'I-Concluding Statement': 11,
 'B-Counterclaim': 5,
 'I-Counterclaim': 12,
 'B-Rebuttal': 6,
 'I-Rebuttal': 13,
 'O': 14,
 'Special': -100}

In [10]:
test_names, train_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/train'))):
    test_names.append(f.replace('.txt', ''))
    train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r', encoding='utf-8').read())
train_text_df = pd.DataFrame({'id': test_names, 'text': train_texts})
train_text_df.head()

100%|██████████| 15594/15594 [00:00<00:00, 27307.37it/s]


,id,text
0,4BCA0580352F,Isn't it wonderful to know that you get a chan...
1,29A8E6DA6F04,"One uses a car to go to the store, pick someon..."
2,17E8BB6C550B,Driverless cars do seem like the thing today's...
3,319B2511943D,Have you ever thought your choice wasn't the b...
4,C9B1827498E4,"Imagine you're sitting at home, watching a sho..."


In [11]:
all_entities = []
all_text_list = []
for ii,i in enumerate(train_text_df.iterrows()):
    if ii%100==0: print(ii,', ',end='')
    total = i[1]['text'].split().__len__()
    text_list = i[1]['text'].split()
    entities = ["O"]*total
    for j in train[train['id'] == i[1]['id']].iterrows():
        discourse = j[1]['discourse_type']
        list_ix = [int(x) for x in j[1]['predictionstring'].split(' ')]
        entities[list_ix[0]] = f"B-{discourse}"
        for k in list_ix[1:]: entities[k] = f"I-{discourse}"
    all_entities.append(entities)
    all_text_list.append(text_list)
train_text_df['entities'] = all_entities
train_text_df['text_list'] = all_text_list

0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 2000 , 2100 , 2200 , 2300 , 2400 , 2500 , 2600 , 2700 , 2800 , 2900 , 3000 , 3100 , 3200 , 3300 , 3400 , 3500 , 3600 , 3700 , 3800 , 3900 , 4000 , 4100 , 4200 , 4300 , 4400 , 4500 , 4600 , 4700 , 4800 , 4900 , 5000 , 5100 , 5200 , 5300 , 5400 , 5500 , 5600 , 5700 , 5800 , 5900 , 6000 , 6100 , 6200 , 6300 , 6400 , 6500 , 6600 , 6700 , 6800 , 6900 , 7000 , 7100 , 7200 , 7300 , 7400 , 7500 , 7600 , 7700 , 7800 , 7900 , 8000 , 8100 , 8200 , 8300 , 8400 , 8500 , 8600 , 8700 , 8800 , 8900 , 9000 , 9100 , 9200 , 9300 , 9400 , 9500 , 9600 , 9700 , 9800 , 9900 , 10000 , 10100 , 10200 , 10300 , 10400 , 10500 , 10600 , 10700 , 10800 , 10900 , 11000 , 11100 , 11200 , 11300 , 11400 , 11500 , 11600 , 11700 , 11800 , 11900 , 12000 , 12100 , 12200 , 12300 , 12400 , 12500 , 12600 , 12700 , 12800 , 12900 , 13000 , 13100 , 13200 , 13300 , 13400 , 13500 , 13600 , 13700 , 13800 , 

In [12]:
print( train_text_df.shape)
train_text_df.head()

(15594, 4)


,id,text,entities,text_list
0,4BCA0580352F,Isn't it wonderful to know that you get a chan...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea...","[Isn't, it, wonderful, to, know, that, you, ge..."
1,29A8E6DA6F04,"One uses a car to go to the store, pick someon...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea...","[One, uses, a, car, to, go, to, the, store,, p..."
2,17E8BB6C550B,Driverless cars do seem like the thing today's...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea...","[Driverless, cars, do, seem, like, the, thing,..."
3,319B2511943D,Have you ever thought your choice wasn't the b...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea...","[Have, you, ever, thought, your, choice, wasn'..."
4,C9B1827498E4,"Imagine you're sitting at home, watching a sho...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea...","[Imagine, you're, sitting, at, home,, watching..."


In [13]:
from datasets import Dataset

ds = Dataset.from_pandas(train_text_df)
datasets = ds.train_test_split(test_size = 0.1, shuffle=True, seed=42)
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'entities', 'text_list'],
        num_rows: 14034
    })
    test: Dataset({
        features: ['id', 'text', 'entities', 'text_list'],
        num_rows: 1560
    })
})

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', add_prefix_space=True)

In [15]:
def fix_beginnings(labels):
    for i in range(1,len(labels)):
        curr_lab = labels[i]
        prev_lab = labels[i-1]
        if curr_lab in range(7,14):
            if prev_lab != curr_lab and prev_lab != curr_lab - 7:
                labels[i] = curr_lab -7
    return labels

def preparing_train_dataset(examples):
    encoding = tokenizer(examples['text_list'], truncation=True, padding=True, max_length = 1024, is_split_into_words=True)
    total= len(encoding['input_ids'])
    encoding['word_ids']=[]
    encoding['labels']=[]
    for i in range(total):
        labels = [l2i['O'] for _ in range(len(encoding['input_ids'][i]))]
        word_idx = encoding.word_ids(batch_index=i)
        for j in range(len(word_idx)):
            if word_idx[j] is None:
                labels[j]=l2i['Special']
            else:
                label = examples['entities'][i][word_idx[j]]
                labels[j]=l2i[label]
        labels = fix_beginnings(labels)
        encoding['labels'].append(labels)
        encoding['word_ids'].append(word_idx)
        
    return encoding

In [16]:
tokenized_datasets = datasets.map(preparing_train_dataset, batched=True, batch_size=10000, remove_columns=datasets["train"].column_names)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoConfig

config = AutoConfig.from_pretrained('allenai/longformer-base-4096')
config.num_labels = N_LABELS
model = AutoModelForTokenClassification.from_pretrained('allenai/longformer-base-4096', config = config)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForTokenClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForTokenClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN

In [18]:
training_args = TrainingArguments(
    output_dir = './output/longformer-baseline',
    evaluation_strategy = 'epoch',
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    gradient_accumulation_steps = 2,
    learning_rate = 5e-5,
    weight_decay = 0.01,
    max_grad_norm = 10,
    num_train_epochs = 10,
    warmup_ratio = 0.1,
    logging_strategy = 'steps',
    logging_steps = 50,
    save_strategy = 'epoch',
    save_total_limit = 1,
    seed = 42,
    # eval_steps = 50,
    # save_steps = 50,
    dataloader_num_workers = 2,
    load_best_model_at_end = True,
    metric_for_best_model = 'f1',# need to fix
    group_by_length = True,
    report_to = 'wandb',
)

In [19]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [20]:
import numpy as np
from datasets import load_metric

metric = load_metric('seqeval')

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    
    true_predictions = [
        [i2l[p] for (p, l) in zip(prediction, label) if l!= -100] for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [i2l[l] for (p, l) in zip(prediction, label) if l!= -100] for prediction, label in zip(predictions, labels)
    ]
    
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [21]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, 
)

In [22]:
import wandb

run = wandb.init(project='Feedback-prize', entity='donggunseo', name='longformer-baseline')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
trainer.train()
run.finish()
trainer.save_model('best_model/longformer-baseline_best')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: quarter100 (use `wandb login --relogin` to force relogin)


***** Running training *****
  Num examples = 14034
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 8770
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,0.705900,0.654413,0.136531,0.266782,0.180624,0.783154
1,0.596400,0.593971,0.228696,0.373329,0.283639,0.799208
2,0.541100,0.590403,0.251668,0.420714,0.314941,0.800573
3,0.482700,0.606960,0.261331,0.432975,0.325937,0.801598
4,0.402200,0.639762,0.275161,0.419328,0.332281,0.799773
5,0.335900,0.665469,0.259798,0.430274,0.323979,0.789490
6,0.283400,0.739562,0.283025,0.431174,0.341734,0.796805
7,0.225300,0.776141,0.278434,0.429581,0.337874,0.791897
8,0.194900,0.836094,0.280119,0.436993,0.341397,0.793263
9,0.155000,0.865031,0.286298,0.431659,0.344264,0.794407


***** Running Evaluation *****
  Num examples = 1560
  Batch size = 8
Saving model checkpoint to ./output/longformer-baseline/checkpoint-877
Configuration saved in ./output/longformer-baseline/checkpoint-877/config.json
Model weights saved in ./output/longformer-baseline/checkpoint-877/pytorch_model.bin
tokenizer config file saved in ./output/longformer-baseline/checkpoint-877/tokenizer_config.json
Special tokens file saved in ./output/longformer-baseline/checkpoint-877/special_tokens_map.json
Deleting older checkpoint [output/longformer-baseline/checkpoint-1400] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1560
  Batch size = 8
Saving model checkpoint to ./output/longformer-baseline/checkpoint-1754
Configuration saved in ./output/longformer-baseline/checkpoint-1754/config.json
Model weights saved in ./output/longformer-baseline/checkpoint-1754/pytorch_model.bin
tokenizer config file saved in ./output/longformer-baseline/checkpoint-1754/tokenizer_config.

eval/accuracy,▁▇██▇▃▆▄▅▅
eval/f1,▁▅▇▇▇▇████
eval/loss,▃▁▁▁▂▃▅▆▇█
eval/precision,▁▅▆▇▇▇████
eval/recall,▁▅▇█▇█████
eval/runtime,▁▂▃▄▁▃▂█▂▄
eval/samples_per_second,█▇▆▅█▆▇▁▇▅
eval/steps_per_second,▇▇▆▅█▅▇▁▇▅
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▂▃▅▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁


Saving model checkpoint to best_model/longformer-baseline_best
Configuration saved in best_model/longformer-baseline_best/config.json
Model weights saved in best_model/longformer-baseline_best/pytorch_model.bin
tokenizer config file saved in best_model/longformer-baseline_best/tokenizer_config.json
Special tokens file saved in best_model/longformer-baseline_best/special_tokens_map.json


Validation

In [37]:
def preparing_valid_dataset(examples):
    encoding = tokenizer(examples['text_list'], truncation=True, padding=True, max_length = 1024, is_split_into_words=True)
    total= len(encoding['input_ids'])
    encoding['labels']=[]
    encoding['word_ids']=[]
    for i in range(total):
        labels = [l2i['O'] for _ in range(len(encoding['input_ids'][i]))]
        word_idx = encoding.word_ids(batch_index=i)
        for j in range(len(word_idx)):
            if word_idx[j] is None:
                labels[j]=l2i['Special']
            else:
                label = examples['entities'][i][word_idx[j]]
                if label[0]=='B' and j!=0 and word_idx[j-1]==word_idx[j]:
                    label_t = label.replace('B', 'I')
                    labels[j]=l2i[label_t]
                else:
                    labels[j]=l2i[label]
        labels = fix_beginnings(labels)
        encoding['labels'].append(labels)
        encoding['word_ids'].append(word_idx)
    return encoding

In [38]:
tokenized_datasets_valid = datasets['test'].map(preparing_valid_dataset, batched=True, batch_size=10000, remove_columns=datasets["test"].column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [40]:
prediction,_,_ = trainer.predict(tokenized_datasets_valid)

The following columns in the test set  don't have a corresponding argument in `LongformerForTokenClassification.forward` and have been ignored: word_ids.
***** Running Prediction *****
  Num examples = 1560
  Batch size = 8


In [41]:
prediction.shape

(1560, 1024, 15)

In [47]:
preds = np.argmax(prediction, axis=-1)
x = []
for k,label_pred in tqdm(enumerate(preds)):
    token_preds = [i2l[i] for i in label_pred]
    y = []
    word_ids = tokenized_datasets_valid['word_ids'][k]
    previous_word_idx = -1
    for idx, word_idx in enumerate(word_ids):
        if word_idx == None:
            continue
        elif word_idx != previous_word_idx:
            y.append(token_preds[idx])
            previous_word_idx = word_idx
    x.append(y)

1560it [23:19,  1.11it/s]


In [60]:
valid_id_list = list(datasets['test']['id'])
df = pd.DataFrame()
for idx in valid_id_list:
    dfx = train.loc[train['id']==idx].reset_index(drop=True)
    df = pd.concat([df, dfx], ignore_index=True)

In [62]:
df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,DE05DCB75D79,1.620742e+12,0.0,398.0,Driverless cars is the way the automotive indu...,Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,DE05DCB75D79,1.620742e+12,399.0,1039.0,Almost all the successful driverless cars have...,Evidence,Evidence 1,68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 8...
2,DE05DCB75D79,1.620742e+12,1040.0,1177.0,The knowledge of how to design a smart car has...,Claim,Claim 1,178 179 180 181 182 183 184 185 186 187 188 18...
3,DE05DCB75D79,1.620742e+12,1178.0,1435.0,Improvments on technology and cheaper technol...,Evidence,Evidence 2,206 207 208 209 210 211 212 213 214 215 216 21...
4,DE05DCB75D79,1.621526e+12,1436.0,1800.0,"People have long dreamed of driverless cars, a...",Evidence,Evidence 3,248 249 250 251 252 253 254 255 256 257 258 25...
...,...,...,...,...,...,...,...,...
14462,CFF21231DFEC,1.622820e+12,398.0,446.0,"Also, people should be able to make urgent calls",Claim,Claim 2,75 76 77 78 79 80 81 82 83
14463,CFF21231DFEC,1.622820e+12,480.0,569.0,people shouldnt be able to use their cellphon...,Claim,Claim 3,91 92 93 94 95 96 97 98 99 100 101 102 103 104
14464,CFF21231DFEC,1.622820e+12,576.0,637.0,it puts not only the driver in danger but othe...,Claim,Claim 4,106 107 108 109 110 111 112 113 114 115 116 117
14465,CFF21231DFEC,1.622820e+12,638.0,743.0,Its also ilegal to be on the phone while your ...,Claim,Claim 5,118 119 120 121 122 123 124 125 126 127 128 12...


In [66]:
final_pred = []
for i in range(len(df)):
    idx = df['id'][i]
    idx_list = list(df['id'].unique())
    pred = x[idx_list.index(idx)]
    j = 0
    while j < len(pred):
        cls = pred[j]
        if cls =='O': 
            j+=1
        else: 
            cls = cls.replace('B', 'I')
        end = j+1
        while end < len(pred) and pred[end] == cls:
            end +=1
        if cls != 'O' and cls != '' and end-j>7:
            final_pred.append((idx, cls.replace('I-', ''), ' '.join(map(str, list(range(j, end))))))
        j = end
        
oof = pd.DataFrame(final_pred)
oof.columns = ['id', 'class', 'predictionstring']

In [67]:
oof

,id,class,predictionstring
0,DE05DCB75D79,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
1,DE05DCB75D79,Evidence,32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 4...
2,DE05DCB75D79,Evidence,248 249 250 251 252 253 254 255 256 257 258 25...
3,DE05DCB75D79,Position,315 316 317 318 319 320 321 322 323 324 325 32...
4,DE05DCB75D79,Concluding Statement,328 329 330 331 332 333 334 335 336 337 338 33...
...,...,...,...
142164,CFF21231DFEC,Claim,118 119 120 121 122 123 124 125 126 127 128
142165,CFF21231DFEC,Claim,129 130 131 132 133 134 135 136 137 138
142166,CFF21231DFEC,Evidence,139 140 141 142 143 144 145 146 147 148 149 15...
142167,CFF21231DFEC,Claim,193 194 195 196 197 198 199 200 201 202 203 20...


In [70]:
def link_evidence(oof):
  if not len(oof):
    return oof
  
  def jn(pst, start, end):
    return " ".join([str(x) for x in pst[start:end]])
  
  thresh = 1
  idu = oof['id'].unique()
  eoof = oof[oof['class'] == "Evidence"]
  neoof = oof[oof['class'] != "Evidence"]
  eoof.index = eoof[['id', 'class']]
  for thresh2 in range(26, 27, 1):
    retval = []
    for idv in tqdm(idu, desc='link_evidence', leave=False):
      for c in ['Evidence']:
        q = eoof[(eoof['id'] == idv)]
        if len(q) == 0:
          continue
        pst = []
        for r in q.itertuples():
          pst = [*pst, -1,  *[int(x) for x in r.predictionstring.split()]]
        start = 1
        end = 1
        for i in range(2, len(pst)):
          cur = pst[i]
          end = i
          if  ((cur == -1) and ((pst[i + 1] > pst[end - 1] + thresh) or (pst[i + 1] - pst[start] > thresh2))):
            retval.append((idv, c, jn(pst, start, end)))
            start = i + 1
        v = (idv, c, jn(pst, start, end + 1))
        retval.append(v)
    roof = pd.DataFrame(retval, columns=['id', 'class', 'predictionstring'])
    roof = roof.merge(neoof, how='outer')
    return roof

oof = link_evidence(oof)

In [71]:
oof

,id,class,predictionstring
0,DE05DCB75D79,Evidence,32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 4...
1,DE05DCB75D79,Evidence,248 249 250 251 252 253 254 255 256 257 258 25...
2,8C4C7081A2AF,Evidence,160 161 162 163 164 165 166 167 168 169 170 17...
3,8C4C7081A2AF,Evidence,219 220 221 222 223 224 225 226 227 228 229 23...
4,8C4C7081A2AF,Evidence,219 220 221 222 223 224 225 226 227 228 229 23...
...,...,...,...
126523,CFF21231DFEC,Claim,193 194 195 196 197 198 199 200 201 202 203 20...
126524,CFF21231DFEC,Claim,193 194 195 196 197 198 199 200 201 202 203 20...
126525,CFF21231DFEC,Claim,193 194 195 196 197 198 199 200 201 202 203 20...
126526,CFF21231DFEC,Claim,193 194 195 196 197 198 199 200 201 202 203 20...


In [72]:
def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]


def score_feedback_comp(pred_df, gt_df):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df[['id','discourse_type','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df = pred_df[['id','class','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')

    joined['overlaps'] = joined.apply(calc_overlap, axis=1)

    # 2. If the overlap between the ground truth and prediction is >= 0.5, 
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])


    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)
    tp_pred_ids = joined.query('potential_TP') \
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','predictionstring_gt']).first()['pred_id'].values

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids]

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    #calc microf1
    my_f1_score = TP / (TP + 0.5*(FP+FN))
    return my_f1_score

In [73]:
f1s=[]
for c in classes:
    pred_df = oof.loc[oof['class']==c].copy()
    gt_df = df.loc[df['discourse_type']==c].copy()
    f1 = score_feedback_comp(pred_df, gt_df)
    print(c,f1)
    f1s.append(f1)
print('Overall', np.mean(f1s))

Lead 0.14584103512014787
Position 0.12865816222116105
Evidence 0.1624319646843412
Claim 0.1001296582876741
Concluding Statement 0.14270352330950725
Counterclaim 0.08195035981788809
Rebuttal 0.06278713629402756
Overall 0.11778597710496386
